Authors: Frank R Pereira, Melroy Pereira

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import dash
import dash_table
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly
from plotly import graph_objects as go

In [2]:
from BackTesterV1 import*
import datetime as dt

In [3]:
#######################################################
data = pd.read_csv("gbpusd.csv")
data = data[['Open', 'High', 'Low', 'Close']]
data
######################################################

,Open,High,Low,Close
0,1.32619,1.32658,1.32509,1.32513
1,1.32513,1.32540,1.32353,1.32394
2,1.32393,1.32472,1.32314,1.32453
3,1.32453,1.32671,1.32428,1.32559
4,1.32559,1.32559,1.32559,1.32559
...,...,...,...,...
2708,1.23252,1.23391,1.23108,1.23282
2709,1.23282,1.23438,1.23193,1.23299
2710,1.23299,1.23302,1.23004,1.23096
2711,1.23096,1.23199,1.23039,1.23084


In [4]:
def TradingSystem(df_):
    d_train = df_.iloc[:-1, :]
    d_test = df_.iloc[-1, :]
    
    sma = d_train['Open'].rolling(window=3).mean().values[-1]

    if (d_test['Open'] > sma):
        pred = -1
    elif (d_test['Open'] < sma):
        pred = 1
    else:
        pred = 0
        
    return (pred)

In [5]:
Bt = BackTester(TradingSystem, data, 3, 1, 10000)
pos = Bt.Backtest()

Dash app running on http://127.0.0.1:9181/


In [6]:
pos

,Open,High,Low,Close,Signal,Pred,Return,P&L
3,1.3245,1.3267,1.3243,1.3256,1,1,110.0,10110.0
4,1.3256,1.3256,1.3256,1.3256,-1,-1,0.0,10110.0
5,1.3256,1.3256,1.3256,1.3256,-1,-1,0.0,10110.0
6,1.3256,1.3256,1.3256,1.3256,-1,-1,0.0,10110.0
7,1.3256,1.3256,1.3256,1.3256,-1,0,0.0,10110.0
...,...,...,...,...,...,...,...,...
2708,1.2325,1.2339,1.2311,1.2328,1,1,30.0,9020.0
2709,1.2328,1.2344,1.2319,1.2330,1,1,20.0,9040.0
2710,1.2330,1.2330,1.2300,1.2310,-1,1,-200.0,8840.0
2711,1.2310,1.2320,1.2304,1.2308,-1,1,-20.0,8820.0


In [7]:
#Number of Trdaes
long_num = len(pos.loc[pos['Pred'] == 1])
short_num = len(pos.loc[pos['Pred'] == -1])
no_trade_num = len(pos.loc[pos['Pred'] == 0])

#Total Long trades and its details
long_trades = pos['Return'].loc[pos.loc[pos['Pred'] == 1].index]
pos_long_trades = len(long_trades[long_trades > 0])
neg_long_trades = len(long_trades[long_trades < 0])
zero_long_trades = len(long_trades[long_trades == 0])

#Total Short trades and its details
short_trades = pos['Return'].loc[pos.loc[pos['Pred'] == -1].index]
pos_short_trades = len(short_trades[short_trades > 0])
neg_short_trades = len(short_trades[short_trades < 0])
zero_short_trades = len(short_trades[short_trades == 0])

report = {'Total Long Trades': [long_num],
         'Total Short Trades': [short_num],
         'Total No trades': [no_trade_num],
         'Total Positive Trades':[pos_long_trades+pos_short_trades],
         'Total Negative Trades':[neg_long_trades+neg_short_trades],
         'Total Positive Long trades': [pos_long_trades],
         'Total Negative Long trades': [neg_long_trades],
         'Total Zero Long trades': [zero_long_trades],
         'Total Positive Short trades': [pos_short_trades],
         'Total Negative Short trades': [neg_short_trades],
         'Total Zero Short trades': [zero_short_trades]}

In [8]:
pd.DataFrame(report)

,Total Long Trades,Total Short Trades,Total No trades,Total Positive Trades,Total Negative Trades,Total Positive Long trades,Total Negative Long trades,Total Zero Long trades,Total Positive Short trades,Total Negative Short trades,Total Zero Short trades
0,990,978,742,944,894,465,451,74,479,443,56
